In [2]:
!pip install flask flask-ngrok flask-cors openai-whisper transformers accelerate bitsandbytes sentencepiece dotenv pyngrok
!apt-get install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 13.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s

In [1]:
from utils import *
from flask import Flask, request, jsonify
from flask_cors import CORS
from flask_ngrok import run_with_ngrok
import whisper
import re, json, os
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from dotenv import load_dotenv
from huggingface_hub.hf_api import HfFolder
from pyngrok import ngrok
from collections import defaultdict

In [2]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")
ng_token = os.getenv("NG_TOKEN")
HfFolder.save_token(hf_token)
ngrok.set_auth_token(ng_token)

In [3]:
public_url = ngrok.connect(5000)
print("🔥 Public URL:", public_url)

🔥 Public URL: NgrokTunnel: "https://7580-34-125-16-213.ngrok-free.app" -> "http://localhost:5000"


In [4]:
config = get_config()
app = Flask(__name__)
CORS(app)
# run_with_ngrok(app)

In [5]:
audio_model = whisper.load_model(config["audio"]["model_name"])
llm_model_name = config["llm"]["model_name"]

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16"
)

tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    device_map="auto",
    quantization_config=bnb_config
)

llm_pipeline = pipeline("text-generation", model=llm_model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [7]:
@app.route("/events", methods=["GET"])
def get_calendar_events():
    calendar_path = "calendar.json"
    if not os.path.exists(calendar_path):
        return jsonify({})

    with open(calendar_path, "r") as f:
        calendar_data = json.load(f)

    print(f"Calendar data: {calendar_data}")

    return jsonify(calendar_data)

In [8]:
@app.route("/transcribe", methods=["POST"])
def transcribe_audio_route():
    file = request.files.get("file")
    if not file:
        return jsonify({"error": "No audio file provided"}), 400

    filepath = "temp_audio.webm"
    file.save(filepath)

    wav_path = "converted_audio.wav"
    os.system(f"ffmpeg -y -i {filepath} -ar 16000 -ac 1 {wav_path}")

    transcribed_text = transcribe_audio(audio_model, wav_path)

    response = llm_pipeline(
        build_prompt(transcribed_text),
        max_new_tokens=config["llm"]["max_new_tokens"],
        do_sample=False
    )[0]["generated_text"]

    json_block = re.search(r'\{.*?\}', response, re.DOTALL)
    json_output = json.loads(json_block.group(0)) if json_block else {}

    # updated_output = update_date_from_message(json_output, transcribed_text)
    updated_output = json_output
    updated_output["transcript"] = transcribed_text

    calendar_path = "calendar.json"
    date_key = updated_output["date"]

    if os.path.exists(calendar_path):
        with open(calendar_path, "r") as f:
            calendar_data = json.load(f)
    else:
        calendar_data = {}

    if date_key not in calendar_data:
        calendar_data[date_key] = []

    calendar_data[date_key].append({
        "person": updated_output["person"],
        "time": updated_output["time"],
        "purpose": updated_output["purpose"],
        "transcript": updated_output["transcript"]
    })

    with open(calendar_path, "w") as f:
        json.dump(calendar_data, f, indent=2)

    return jsonify(updated_output)

In [ ]:
app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
